In [6]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [3]:
import ultralytics
from tqdm import tqdm

In [31]:
import os
import csv
import cv2

def convert_csv_to_yolo_seg(csv_path, output_labels_dir, class_mapping, dataset_base_path):
    """
    Convert CSV annotations to YOLOv8-seg format.
    
    Args:
        csv_path (str): Path to the CSV file containing annotations.
        output_labels_dir (str): Directory to save YOLO format label files.
        class_mapping (dict): Mapping of class names to class IDs.
        dataset_base_path (str): Base path to prepend to relative paths in the CSV.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_labels_dir, exist_ok=True)

    with open(csv_path, "r") as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            # Construct absolute paths
            image_path = os.path.join(dataset_base_path, row["image_path"])
            mask_path = os.path.join(dataset_base_path, row["mask_path"])
            label = row["label"]

            # Get image dimensions
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Image not found at {image_path}")
                continue
            image_height, image_width = image.shape[:2]

            # Read mask (assuming mask is a binary image)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"Warning: Mask not found at {mask_path}")
                continue

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Prepare YOLO-seg annotation
            yolo_annotation = []
            for contour in contours:
                # Simplify contour to polygon points
                polygon_points = contour.squeeze().tolist()
                if len(polygon_points) < 3:  # Skip invalid polygons
                    continue

                # Normalize polygon points
                normalized_points = []
                for x, y in polygon_points:
                    normalized_x = x / image_width
                    normalized_y = y / image_height
                    normalized_points.extend([normalized_x, normalized_y])

                # Add class ID and normalized points to annotation
                yolo_annotation.append(f"{class_mapping[label]} " + " ".join(map(str, normalized_points)))

            # Save YOLO-seg annotation to file
            if yolo_annotation:
                output_txt_path = os.path.join(output_labels_dir, os.path.basename(image_path).replace(".jpg", ".txt"))
                with open(output_txt_path, "w") as f:
                    f.write("\n".join(yolo_annotation))

# Example usage
dataset_base_path = "C:\\Users\\Ibrahim_Hegazi\\Desktop\\Graduation_Project\\dataset"
csv_path = os.path.join(dataset_base_path, "labels.csv")
output_labels_dir = os.path.join(dataset_base_path, "labels")
class_mapping = {'angular_leafspot':0,
'anthracnose_fruit_rot':1,
'blossom_blight':2,
'gray_mold':3,
'leaf_spot':4,
'powdery_mildew_fruit':5,
'powdery_mildew_leaf':6
}  # Map labels to class IDs

convert_csv_to_yolo_seg(csv_path, output_labels_dir, class_mapping, dataset_base_path)

In [45]:
import os

dataset_path = "C:\\Users\\Ibrahim_Hegazi\\Desktop\\Graduation_Project\\dataset"
if not os.access(dataset_path, os.R_OK | os.W_OK):
    print(f"Permission denied: {dataset_path}")
else:
    print(f"Access granted: {dataset_path}")

Access granted: C:\Users\Ibrahim_Hegazi\Desktop\Graduation_Project\dataset


# Script for Stratified Split


In [75]:
import os
import random
import shutil
from collections import defaultdict

def split_dataset(dataset_path, train_ratio=0.8):
    images_dir = os.path.join(dataset_path, "images")
    labels_dir = os.path.join(dataset_path, "labels")
    train_dir = os.path.join(dataset_path, "train")
    val_dir = os.path.join(dataset_path, "val")

    # Create train and val directories
    os.makedirs(os.path.join(train_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(train_dir, "labels"), exist_ok=True)
    os.makedirs(os.path.join(val_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(val_dir, "labels"), exist_ok=True)

    # Group images by class
    class_to_images = defaultdict(list)
    image_files = [f for f in os.listdir(images_dir) if f.endswith(".jpg")]

    for img_file in image_files:
        label_file = img_file.replace(".jpg", ".txt")
        label_path = os.path.join(labels_dir, label_file)
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                lines = f.readlines()
                if lines:
                    # Assuming the first line contains the class ID
                    class_id = lines[0].split()[0]
                    class_to_images[class_id].append((img_file, label_file))

    # Split each class into train and val
    for class_id, images in class_to_images.items():
        random.shuffle(images)
        split_idx = int(len(images) * train_ratio)
        train_images = images[:split_idx]
        val_images = images[split_idx:]

        # Move train images and labels
        for img_file, label_file in train_images:
            shutil.move(os.path.join(images_dir, img_file), os.path.join(train_dir, "images", img_file))
            shutil.move(os.path.join(labels_dir, label_file), os.path.join(train_dir, "labels", label_file))

        # Move val images and labels
        for img_file, label_file in val_images:
            shutil.move(os.path.join(images_dir, img_file), os.path.join(val_dir, "images", img_file))
            shutil.move(os.path.join(labels_dir, label_file), os.path.join(val_dir, "labels", label_file))

# Example usage
dataset_path = "C:\\Users\\Ibrahim_Hegazi\\Desktop\\Graduation_Project\\dataset"
split_dataset(dataset_path)

In [3]:
%cd C:\\Users\\Ibrahim_Hegazi\\Desktop\\Graduation_Project\\dataset

C:\Users\Ibrahim_Hegazi\Desktop\Graduation_Project\dataset


In [5]:
import os

# List all files and directories in the current directory
files = os.listdir()

# Print the files
for file in files:
    print(file)

annotations
dataset.yaml
labels.csv
masks
runs
train
val
yolov8n-seg.pt


# Training the Model

## Training the model Could not be done here due to my hardware capabilities, I will run the Training Code on Kaggle Notebooks and i will provide the code later on in another notebook if you are intersted